In [1]:
%%capture
!mamba install cudatoolkit xformers bitsandbytes pytorch pytorch-cuda=11.8 \
    -c pytorch -c nvidia -c xformers -c conda-forge -y
!pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git"
!pip uninstall datasets -y
!pip install datasets

!pip install "git+https://github.com/huggingface/transformers.git" # Native 4bit loading works!

In [2]:
from unsloth import FastLanguageModel

/usr/local/lib/python3.10/dist-packages/unsloth/__init__.py:71: UserWarning: Unsloth: Running `ldconfig /usr/lib64-nvidia` to link CUDA.
  warnings.warn(


# Base model

In [3]:
MAX_SEQ_LENGTH = 2048
dtype=None
load_in_4bit=True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="chillies/IELTS-fighter",
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=dtype,
    load_in_4bit=load_in_4bit
)

adapter_config.json:   0%|          | 0.00/703 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Mistral patching release 2024.3
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.22.post7. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:155: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/336M [00:00<?, ?B/s]

Unsloth 2024.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# DPO

In [4]:
MAX_SEQ_LENGTH = 2048
dtype=None
load_in_4bit=True

dpo_model, dpo_tokenizer = FastLanguageModel.from_pretrained(
    model_name="chillies/DPO_ielts_fighter",
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=dtype,
    load_in_4bit=load_in_4bit
)

adapter_config.json:   0%|          | 0.00/703 [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Mistral patching release 2024.3
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.22.post7. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:155: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


adapter_model.safetensors:   0%|          | 0.00/336M [00:00<?, ?B/s]

# Inference

In [5]:
from IPython.display import display, Markdown

def display_markdown(markdown_string):
    markdown_string = markdown_string.replace("<\s>", "")
    markdown_string = markdown_string.replace("<s>", "")
    display(Markdown(markdown_string))

In [6]:
PROMPT = """
In this task, you are required to evaluate an IELTS Writing Task 2 essay. Consider the following four criteria and provide a detailed assessment for each, along with a suggested band score:

## Task Achievement:
- Evaluate how well the candidate has addressed the given task.
- Assess the clarity and coherence of the response in presenting ideas.
- Identify if the candidate has fully covered all parts of the task and supported arguments appropriately.
- Suggested Band Score (Task Achievement): [Insert Score]

## Coherence and Cohesion:
- Assess the overall organization and structure of the essay.
- Evaluate the use of linking devices to connect ideas and paragraphs.
- Identify if there is a logical flow of information.
- Suggested Band Score (Coherence and Cohesion): [Insert Score]

## Lexical Resource (Vocabulary):
- Examine the range and accuracy of vocabulary used in the essay.
- Point out specific mistakes in vocabulary, such as inaccuracies or overuse of certain words and Suggest modified versions or alternatives for the identified mistakes. [list of mistakes and rectify]
- Assess the appropriateness of vocabulary for the given context.
- Suggested Band Score (Lexical Resource): [Insert Score]

## Grammatical Range and Accuracy:
- Evaluate the variety and complexity of sentence structures.
- Point out specific grammatical errors, such as incorrect verb forms or sentence construction and Suggest modified versions or corrections for the identified mistakes. [list of mistakes and rectify]
- Examine the use of punctuation and sentence formation.
- Suggested Band Score (Grammatical Range and Accuracy): [Insert Score]

## Overall Band Score:

- Provide an overall band score for the essay, considering the holistic performance across all criteria.
- Consider the synergy of the essay in meeting the task requirements cohesively.
- Suggested Overall Band Score: [Insert Score]

## Feedback and Additional Comments:
- Provide constructive feedback highlighting specific strengths and areas for improvement.
- Suggest strategies for enhancement in weaker areas.

## Prompt:
{}

## Essay:
{}

## Evaluation:
{}"""

In [21]:
def generate_evaluation(model, tokenizer, prompt_essay, essay):
    # alpaca_prompt = Copied from above
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference
    inputs = tokenizer(
    [
        PROMPT.format(
            prompt_essay,
            essay,
            "",
        )
    ], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 2048, use_cache = True, temperature=0.8, do_sample=True)
    answer = tokenizer.batch_decode(outputs)
    evaluation = answer[0].split("Evaluation:")[1]

    return (prompt_essay, essay, evaluation)

**Example 1**

In [19]:
p1 = """
The average standard of people's health is likely to be lower in the future than it is now.

To what extent do you agree or disagree with this statement?"""
e1 = """
Due to the development in the fast food culture and the lifestyle, people’s lives are deteriorating day by day. It is said that an average person's health would worsen in the future. In my opinion, I agree with this statement considering how people eat and have a completely different lifestyle than compared in the past.

One of the evident reasons why an individual's health will lower in the future is because of the kind of food they consume today. For example, foods like burgers, pizzas, burritos and milkshakes have no nutrients but are full of carbohydrates and sugar. Eating in large quantities frequently might increase the risk of causing several problems like obesity, cardiovascular diseases and even stroke. The main reason that a person prefers fast food over home-cooked food is due to the easier availability and cheaper cost of  fast food.The government can impose certain taxes on the foods that are unhealthy to people which in turn makes them choose healthier food.

Another reason might be due to the declining interest in sports nowadays. This is because people these days prefer staying indoors. For instance, playing video games and chatting online. This in turn makes them lazy and due to the lack of exercise, they might develop certain complications. In contrast, even though there are many drawbacks to health there is a significant improvement in medical technologies nowadays which can cure diseases ensuring a longer life for  people.

In conclusion,Although there are many technologies available to help improve a person's life there are certain reasons like opting for fast foodfood and not staying active will make an individual's health worse in the future.
"""

In [20]:
(_, _, evaluation_base_1) = generate_evaluation(model, tokenizer, p1, e1)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [22]:
(_, _, evaluation_dpo_1) = generate_evaluation(dpo_model, dpo_tokenizer, p1, e1)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [23]:
display_markdown(evaluation_base_1)


## Task Achievement:
- The candidate has addressed the given task by providing a clear stance on the topic and supporting their arguments with specific examples.
- The response is coherent and presents ideas in a logical order.
- The candidate has fully covered all parts of the task and supported their arguments appropriately.
- Suggested Band Score (Task Achievement): 7

## Coherence and Cohesion:
- The essay is well-organized and structured, with clear transitions between paragraphs.
- Linking devices are used effectively to connect ideas and ensure a smooth flow of information.
- The logical flow of information is maintained throughout the essay.
- Suggested Band Score (Coherence and Cohesion): 7

## Lexical Resource (Vocabulary):
- The candidate demonstrates a good range of vocabulary, using appropriate and varied words to express their ideas.
- There are a few minor inaccuracies in vocabulary, such as "deteriorating" instead of "declining" and "complications" instead of "health issues."
- The vocabulary is generally appropriate for the given context.
- Suggested Band Score (Lexical Resource): 6.5

## Grammatical Range and Accuracy:
- The candidate demonstrates a variety of sentence structures, including complex and compound sentences.
- There are a few minor grammatical errors, such as "an individual's health will lower in the future" instead of "an individual's health will deteriorate in the future" and "In contrast, even though there are many drawbacks to health there is a significant improvement in medical technologies nowadays which can cure diseases ensuring a longer life for people."
- The use of punctuation and sentence formation is generally accurate.
- Suggested Band Score (Grammatical Range and Accuracy): 6.5

## Overall Band Score:
- The essay demonstrates a good understanding of the task and presents a clear and coherent response.
- The candidate uses a range of vocabulary and sentence structures, but there are a few minor inaccuracies in vocabulary and grammar.
- The overall performance across all criteria is strong, and the essay meets the requirements of the task.
- Suggested Overall Band Score: 7

## Feedback and Additional Comments:
- The essay is well-written and demonstrates a good understanding of the topic.
- The candidate could improve their vocabulary by using more precise and varied words.
- The candidate could also improve their grammar by carefully checking for errors in verb forms and sentence construction.
- Overall, the essay is a good example of a well-written IELTS Writing Task 2 essay.</s>

In [24]:
display_markdown(evaluation_dpo_1)



## Task Achievement
- The candidate has addressed the given task and provided a clear stance on the statement.
- The response is coherent in presenting ideas, but lacks specific examples and evidence to support the argument.
- The candidate has not fully covered all parts of the task, as they have not considered the potential positives of future health.
- Suggested Band Score (Task Achievement): 6.5

## Coherence and Cohesion
- The essay is organized into paragraphs, but the flow of information is not always logical.
- Linking devices are used, but they are not always appropriate for the context.
- The essay lacks a clear transition between the paragraphs, making the flow of ideas difficult to follow.
- Suggested Band Score (Coherence and Cohesion): 6.5

## Lexical Resource (Vocabulary)
- The candidate has used a limited range of vocabulary, relying on simple words and phrases.
- There are some inaccuracies in vocabulary, such as "individual's health would lower" (should be "will decrease").
- The vocabulary is generally appropriate for the context, but could benefit from more precise and varied word choices.
- Suggested Band Score (Lexical Resource): 6.0

## Grammatical Range and Accuracy
- The candidate has used a variety of sentence structures, but there are some grammatical errors.
- There are incorrect verb forms, such as "is said" (should be "is stated").
- The sentence construction is sometimes awkward, making the meaning difficult to understand.
- The use of punctuation is also inconsistent.
- Suggested Band Score (Grammatical Range and Accuracy): 6.0

## Overall Band Score

- The essay demonstrates a basic understanding of the task and presents a clear stance, but lacks evidence and coherence.
- The vocabulary is limited and inaccurate, and the grammar is also weak.
- The essay does not fully address all parts of the task and does not thoroughly support the argument.
- Suggested Overall Band Score: 6.0

## Feedback and Additional Comments

- Strengths: The essay has a clear stance and is well-organized into paragraphs.
- Areas for improvement: The candidate should provide more specific examples and evidence to support their argument. They should also improve their vocabulary and grammar. They should also work on improving the flow of ideas and the use of linking devices.
- Suggestions for enhancement: The candidate can improve their vocabulary by using a thesaurus or dictionary. They can also practice writing essays with a focus on specific parts of the task. Additionally, they can read more widely to improve their grammar and sentence construction.</s>

**Example 2**

In [25]:
p2 = """
Many people go to university for academic study. More people should be encouraged to do vocational training because there is a lack of qualified workers such as electricians and plumbers. To what extent do you agree or disagree?"""
e2 = """
Some argued that vocational training should be encouraged for enrollment to produce sufficient future competent labourers in the context of an increasing number of people taking part in the academic field. I completely agree with this opinion, as I believe qualified researchers and workers are equally important and their numbers should be balanced.

Firstly, lacking of competent workers academic study performed by students and researchers in universities yields theoretical results, which are foundations for practical field-work performed by workers. This is a process called "from theory to practice" in which various researchers and workers are involved and how most of the products we are using nowadays went from an idea into reality.  Therefore, a lack of qualified workers would hinder this process and bringing scientific ideas to practical use would be impossible.  For example, the invention of electricity in the 19th was based purely on theoretical knowledge of physics and electric currents. Subsequently, an occupation called "electricians" was born to meet the demands of applying this invention to domestic use in households, such as installing and fixing broken electrical appliances. As physicists only work with theory and cannot handle electric cables or plugs, too many of them and few electricians would make the society impossible to function.

Secondly, schools should take responsibility for integrating vocational training alongside academic study ever since elementary education. This would provide students with necessary hands-on skills in the future and give them an occupation orientation. For example, a student should learn both theoretical physics and how to fix electrical appliances, which is an effective way for them to apply academic knowledge to solving practical problems.

In summary, I believe vocational training is needed for the future sufficiency of competent workers. The balance of both positions would effectively maintain the society's function."""

In [27]:
(_, _, evaluation_base_2) = generate_evaluation(model, tokenizer, p2, e2)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [28]:
(_, _, evaluation_dpo_2) = generate_evaluation(dpo_model, dpo_tokenizer, p2, e2)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [29]:
display_markdown(evaluation_base_2)



## Task Achievement:

The candidate has addressed the task by expressing their agreement with the prompt and providing reasons to support their position. The response is clear and coherent, presenting ideas in a logical manner. The candidate has fully covered all parts of the task and supported their arguments appropriately, by providing specific examples and explanations.

Suggested Band Score (Task Achievement): 7.5

## Coherence and Cohesion:

The essay is well-organized, with a clear introduction, body paragraphs, and conclusion. The candidate has used linking devices to connect ideas and paragraphs, maintaining a logical flow of information. The use of transition words and phrases ensures a smooth progression of thought throughout the essay.

Suggested Band Score (Coherence and Cohesion): 8.0

## Lexical Resource (Vocabulary):

The candidate has demonstrated a good range of vocabulary, using a variety of words and phrases to express their ideas. However, there are a few instances of inaccurate or overused vocabulary. For example, the use of "sufficient" instead of "adequate" could be more precise. Additionally, the use of "integrating" instead of "incorporating" could be more specific.

Suggested Band Score (Lexical Resource): 7.0

## Grammatical Range and Accuracy:

The candidate has used a variety of sentence structures, demonstrating a good range of grammatical knowledge. However, there are a few grammatical errors, such as the use of "subsequently" instead of "subsequently" and the use of "is" instead of "are." Additionally, the use of "should take responsibility for integrating" could be improved by using the passive voice.

Suggested Band Score (Grammatical Range and Accuracy): 7.0

## Overall Band Score:

The essay demonstrates a good overall performance, meeting the task requirements and presenting ideas in a coherent and logical manner. However, there are some areas for improvement in terms of vocabulary precision and grammatical accuracy.

Suggested Overall Band Score: 7.5

## Feedback and Additional Comments:

**Strengths:**

- Clear and coherent organization
- Well-supported arguments
- Good variety of vocabulary

**Areas for improvement:**

- Improve vocabulary precision
- Correct grammatical errors
- Enhance sentence structure and complexity

**Suggestions for enhancement:**

- Use more precise and specific vocabulary, such as "adequate" instead of "sufficient"
- Proofread and check for grammatical errors before submitting the essay
- Practice using a wider range of sentence structures to improve grammatical accuracy</s>

In [30]:
display_markdown(evaluation_dpo_2)


## Task Achievement:
- The candidate has adequately addressed the task by exploring the importance of vocational training in producing qualified workers and providing appropriate examples.
- The response is clear and coherent, with a logical flow of ideas.
- The candidate has fully covered all aspects of the task and provided relevant supporting arguments.
- Suggested Band Score (Task Achievement): 7.5

## Coherence and Cohesion:
- The essay is well-organized and structured, with a clear introduction, body paragraphs, and conclusion.
- The ideas are connected using appropriate linking devices, such as "Firstly" and "Therefore."
- There is a logical flow of information, with each paragraph building on the previous one.
- Suggested Band Score (Coherence and Cohesion): 7.5

## Lexical Resource (Vocabulary):
- The candidate demonstrates a good range of vocabulary, using appropriate words such as "competent" and "qualified."
- However, there are a few inaccuracies and overused words, such as "lack" and "invention."
- The vocabulary is generally appropriate for the context.
- Suggested Band Score (Lexical Resource): 7.0

## Grammatical Range and Accuracy:
- The essay exhibits a variety of sentence structures, including complex and compound sentences.
- However, there are some grammatical errors, such as "the invention of electricity in the 19th was based purely on theoretical knowledge of physics and electric currents." (should be "the invention of electricity in the 19th century was based purely on theoretical knowledge of physics and electric currents.")
- Punctuation and sentence formation are generally accurate.
- Suggested Band Score (Grammatical Range and Accuracy): 7.0

## Overall Band Score:
- The essay demonstrates a good understanding of the task and provides a coherent and well-structured response.
- The candidate uses a range of vocabulary and grammar, but there are some inaccuracies and errors.
- The overall band score for this essay is 7.5.

## Feedback and Additional Comments:

**Strengths:**
- Clear and coherent organization
- Relevant supporting arguments
- Appropriate use of linking devices

**Areas for improvement:**
- Accuracy in vocabulary
- Grammatical errors
- Incorporating more specific examples to support arguments

**Strategies for enhancement:**
- Practice using a wider range of vocabulary, including more sophisticated words and phrases.
- Proofread carefully to identify and correct grammatical errors.
- Use specific examples to strengthen arguments and provide more detail.</s>

**Example 3**

In [33]:
p3 = """
Governments should spend money on railway rather than roads.

To what extent do vou agree or disagree with the statement?
"""
e3 = """
Nowadays, governments have to spend money on railways rather than roads. Some people assert that the installation of a new railway is more crucial. I support this idea because the installation of the railway is more helpful in promoting the suburban area's economy.

Firstly, a railway should be installed because it would make it more convenient for people to travel and commute. For example, people who do not have a car have difficulties commuting. If there is no railway, they should in case of traffic jams or could have to move their home nearby workplaces. Therefore, it is clear that the development of railways has to be done as soon as possible because the comfortable transportation services will encourage people to live in that area, and it is entirely understandable that people usually would like to live in a well-facilitated area.

Next, railways should be installed quickly because installing the railway takes more time than forming roads. Moreover, it promotes the economy of surrounding areas as there will be many facilities such as cafes, restaurants, and hospitals nearby railway stations.

To sum up, it is clear that railway construction should be carried out earlier than roads as convenient transportation will attract a number of tourists. Therefore, the economy of that region will be more robust, and the living costs of that area will also increase.
"""

In [42]:
(_, _, evaluation_based_3) = generate_evaluation(model, tokenizer, p3, e3)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [58]:
(_, _, evaluation_dpo_3) = generate_evaluation(dpo_model, dpo_tokenizer, p3, e3)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [44]:
display_markdown(evaluation_based_3)


## Task Achievement:
- The candidate has addressed the task appropriately by providing reasons for their agreement with the statement that governments should spend money on railways rather than roads.
- The response is clear and coherent, presenting ideas in a logical order.
- The candidate has fully covered all parts of the task and supported arguments appropriately.
- **Suggested Band Score (Task Achievement): 7**

## Coherence and Cohesion:
- The essay is well-organized with clear transitions between paragraphs.
- Linking devices are used effectively to connect ideas and ensure a logical flow of information.
- **Suggested Band Score (Coherence and Cohesion): 7**

## Lexical Resource (Vocabulary):
- The candidate demonstrates a good range of vocabulary, using appropriate words and phrases.
- There are a few instances of inaccurate vocabulary usage, such as "making it more convenient for people to travel and commute" (suggesting that railways are merely necessary rather than beneficial) and "making it more robust" (suggesting that railways directly increase the economy rather than creating opportunities for economic growth).
- **Suggested Band Score (Lexical Resource): 6.5**

## Grammatical Range and Accuracy:
- The candidate demonstrates a variety of sentence structures, including complex sentences.
- There are a few grammatical errors, such as "it would make it more convenient for people to travel and commute" (missing "for them") and "it is clear that the development of railways has to be done as soon as possible" (incorrect use of "it" and "has to").
- **Suggested Band Score (Grammatical Range and Accuracy): 6.5**

## Overall Band Score:
- Considering the holistic performance across all criteria, the suggested overall band score is 7.
- The essay effectively addresses the task, demonstrates coherence and cohesion, and uses a good range of vocabulary.
- The grammatical range and accuracy could be improved.

## Feedback and Additional Comments:

**Strengths:**
- Clear and well-organized structure
- Effective use of linking devices
- Good range of vocabulary

**Areas for Improvement:**
- Accuracy of vocabulary usage
- Grammatical errors
- More in-depth analysis of the topic</s>

In [59]:
display_markdown(evaluation_dpo_3)



## Task Achievement:

- The candidate has partially addressed the task by mentioning the importance of railway construction in promoting the economy of surrounding areas and making it more convenient for people to travel and commute.
- The response is clear and coherent, but some ideas are presented briefly, and the arguments are not fully developed.
- The candidate has not fully covered all parts of the task, as they have not addressed the potential benefits of road development.
- Suggested Band Score (Task Achievement): 6.5

## Coherence and Cohesion:

- The essay lacks overall organization and structure, as the paragraphs do not flow logically.
- The use of linking devices is limited, and the ideas are not effectively connected.
- There is a lack of logical flow of information, and the essay lacks coherence.
- Suggested Band Score (Coherence and Cohesion): 6.0

## Lexical Resource (Vocabulary):

- The essay exhibits a limited range of vocabulary, with some inaccuracies and overuse.
- Mistake 1: "comfortable transportation services" -> "convenient transportation services"
- Mistake 2: "robust economy" -> "thriving economy"
- The vocabulary used is appropriate for the given context, but the essay could benefit from a wider range of words.
- Suggested Band Score (Lexical Resource): 6.5

## Grammatical Range and Accuracy:

- The essay exhibits a limited range of sentence structures, with some grammatical errors.
- Mistake 1: "it is clearly that" -> "it is clear that"
- Mistake 2: "therefore, the economy of that region will be more robust" -> "therefore, the economy of that region will be more robust"
- The use of punctuation and sentence formation is generally accurate, but there are some areas where the grammar could be improved.
- Suggested Band Score (Grammatical Range and Accuracy): 6.0

## Overall Band Score:

- The overall band score for the essay is 6.5.
- The essay demonstrates a limited understanding of the task and provides a partially coherent response.
- The essay exhibits a limited range of vocabulary and grammatical structures.
- While there are some strengths, such as the clear and concise writing style, the essay lacks depth and comprehensiveness.

## Feedback and Additional Comments:

- Strengths:
  - Clear and concise writing style
  - Some valid points are made about the benefits of railway construction
- Areas for improvement:
  - Improve the overall organization and structure of the essay
  - Expand on the arguments and fully address all parts of the task
  - Enhance vocabulary and use a wider range of grammatical structures
  - Proofread carefully to eliminate grammatical errors and improve sentence formation</s>

**Example 4**

In [61]:
p4 = """In many countries, people are moving away from rural areas and towards urban areas. Why do you think that is? What problems can this cause?"""
e4 = """
It is true that advances in the realm of urban areas is better than rural areas. Therefore, the majority of people prefer to reside in town in recent years. The main problem may lie in strain of infrastructure and health problems.
To begin with, increase in the level of people in urban areas can seriously jeopardize the environment. This is because, as more people move into cities, the demand for housing, transportation and utilize increases, often capacity of existing systems. This can lead to overcrowding and shortage of resources like water and electricity. In order to mitigate this concern, individuals should reduce their impact on infrastructure. They can use public transportation instead of driving alone, or they can reduce energy consumption by using energy-efficient appliances and turning off lights when not in use. In doing so, individuals can contribute to the overall effort to reduce the strain of urban infrastructure caused by overpopulation.
Moreover, another worrying aspect of the uprise of people in urban area is health issues. In other words overcrowding and pollution can lead to health problems such as respiratory problems and increased risk of disease outbreaks. To illustrate, in cities with high levels of traffic and industrial emissions residents may be exposed to harmful pollutants that can cause respiratory conditions. Addressing this issue involves mass media shouldering the responsibility to reduce air pollution such as promoting the use of public transportation.
To conclude, living in urban areas can lead to numerous problems. Two main dangers of this switch can be strain of infrastructure and health problems which should be minimized by government and mass media.
"""

In [51]:
(_, _, evaluation_base_4) = generate_evaluation(model, tokenizer, p4, e4)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [62]:
(_, _, evaluation_dppo_4) = generate_evaluation(dpo_model, dpo_tokenizer, p4, e4)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [55]:
display_markdown(evaluation_base_4)


## Task Achievement:
The candidate has addressed the task by discussing the reasons for the shift towards urban areas and the problems that can arise from this trend. The response is clear and coherent, with arguments supported by examples. However, the essay could have addressed the task more fully by examining other potential consequences of urbanization, such as loss of cultural heritage or increased social inequality.

Suggested Band Score (Task Achievement): 7

## Coherence and Cohesion:
The essay is well-organized, with a clear introduction, body paragraphs, and conclusion. However, the essay could benefit from stronger transitions between paragraphs to improve the logical flow of ideas.

Suggested Band Score (Coherence and Cohesion): 6.5

## Lexical Resource (Vocabulary):
The candidate demonstrates a good range of vocabulary, using various words to describe the reasons for and consequences of urbanization. However, there are some inaccuracies and overuse of certain words. For example, "overcrowding" is used several times, and "environment" could be replaced with more specific terms such as "air quality" or "green spaces."

## Mistakes and Rectifications:
- "overcrowding" -> "overpopulation"
- "environment" -> "air quality"

Suggested Band Score (Lexical Resource): 6

## Grammatical Range and Accuracy:
The essay exhibits a variety of sentence structures, but there are some grammatical errors. For instance, the sentence "This is because, as more people move into cities, the demand for housing, transportation and utilize increases" is grammatically incorrect. The verb "utilize" should be replaced with "increase."

Suggested Band Score (Grammatical Range and Accuracy): 6

## Overall Band Score:
Considering the holistic performance across all criteria, the essay merits an overall band score of 6.5. The essay effectively addresses the task, presents a coherent and well-organized structure, and demonstrates a good range of vocabulary. However, the grammatical accuracy and cohesion could be improved, and the lexical resource could benefit from a more precise and varied vocabulary.

## Feedback and Additional Comments:
- Strengthen the transitions between paragraphs to improve the logical flow of ideas.
- Replace inaccurate or overused vocabulary with more specific and varied terms.
- Pay closer attention to grammatical accuracy, particularly in complex sentence structures.
- Include more discussion on the potential benefits of urbanization, such as increased access to infrastructure and services.</s>

In [63]:
display_markdown(evaluation_dppo_4)



## Task Achievement:
The candidate has addressed the prompt by discussing the reasons for people moving to urban areas and the problems that this can cause. However, the response is not fully developed and lacks clear support for the arguments.

The essay does not fully cover all parts of the task. It does not explicitly state the problems that can be caused by urbanization, but rather suggests them without providing specific examples or evidence. Additionally, the essay lacks a concluding paragraph that summarizes the main points and restates the thesis statement.

Suggested Band Score (Task Achievement): 6.0

## Coherence and Cohesion:
The essay is organized into paragraphs, but the overall structure is not clear and the flow of information is not always logical. The transitions between paragraphs are not effectively used to connect ideas and create a cohesive narrative.

The use of linking devices is limited and there is no clear progression of ideas throughout the essay. The paragraphs do not always build on each other or provide a natural transition to the next point.

Suggested Band Score (Coherence and Cohesion): 5.5

## Lexical Resource (Vocabulary):
The essay demonstrates a limited range of vocabulary and there are some inaccuracies in word choice. For example, the phrase "level of people" is not grammatically correct and should be replaced with "population".

The essay does not use a variety of vocabulary to express ideas. The sentences are often repetitive and lack specificity. Additionally, there are some inaccuracies in vocabulary, such as the use of "infrastructure" to refer to urban planning and development.

**Mistakes and Rectifications:**
- "level of people" -> "population"
- "infrastructure" -> "urban planning and development"

Suggested Band Score (Lexical Resource): 5.5

## Grammatical Range and Accuracy:
The essay demonstrates a limited range of grammatical structures and there are numerous grammatical errors throughout the text.

The sentence construction is often simplistic and lacks complexity. The essay contains many grammatical errors, including incorrect verb forms and sentence construction. For example, "This is because" should be replaced with "This is due to" and "they can use public transportation instead of driving alone" should be rephrased as "they can use public transportation or carpooling to reduce their commute".

**Mistakes and Rectifications:**
- "This is because" -> "This is due to"
- "they can use public transportation instead of driving alone" -> "they can use public transportation or carpooling to reduce their commute"

Suggested Band Score (Grammatical Range and Accuracy): 5.0

## Overall Band Score:
The essay meets the task requirements to some extent, but the overall performance is weak. The essay lacks clarity, coherence, and grammatical accuracy.

The essay demonstrates a limited range of vocabulary and grammatical structures. The sentence construction is often simplistic and the transitions between paragraphs are not effective. Additionally, there are numerous grammatical errors throughout the text.

Suggested Overall Band Score: 5.5

## Feedback and Additional Comments:
**Strengths:** The essay addresses the prompt and provides some examples of the problems that can be caused by urbanization.

**Areas for Improvement:**
- Improve the clarity and coherence of the essay by using clear transitions between paragraphs and providing a logical flow of information.
- Expand the range of vocabulary and use more specific and precise terms to express ideas.
- Correct the grammatical errors throughout the essay to improve the overall accuracy and clarity of the writing.
- Provide more specific examples and evidence to support the arguments.
- Include a concluding paragraph that sums up the main points and restates the thesis statement.</s>